# Welcome to Ray on Vertex AI
<div class='markdown-google-sans'>

  ## **Getting started**
</div>

Import Vertex AI SDK for Python, and initialize connection with the Ray cluster on Vertex AI.

In [ ]:
! pip install "google-cloud-aiplatform>=1.51" ray

In [ ]:
import ray

from google.cloud import aiplatform
from google.cloud.aiplatform import vertex_ray

aiplatform.init(project='jkwng-vertex-playground', location='us-central1')

In [ ]:
from ray.runtime_env import RuntimeEnv

runtime_env = RuntimeEnv(
  pip=["torch"],
)

# Initialize connection to the Ray cluster on Vertex AI.
#ray.init(
#    address='vertex_ray://projects/205512073711/locations/us-central1/persistentResources/ray-cluster',
#    runtime_env=runtime_env
#)
ray.init(
    address="ray://10.30.0.223:10001",
    runtime_env=runtime_env
)

SIGTERM handler is not set because current thread is not the main thread.


Python version:,3.10.13
Ray version:,2.9.3
Dashboard:,http://10.130.0.2:8265


In [ ]:
from pprint import pprint

for node in ray.nodes():
    pprint(node)

{'Alive': True,
 'Labels': {'ray.io/node_id': '0932306ef17f5ff3627555a75cdafb6e81d9151d723eae02dee33bab'},
 'MetricsExportPort': 8080,
 'NodeID': '0932306ef17f5ff3627555a75cdafb6e81d9151d723eae02dee33bab',
 'NodeManagerAddress': '10.130.0.2',
 'NodeManagerHostname': 'raycluster-complete-head-b8x7v',
 'NodeManagerPort': 43929,
 'NodeName': '10.130.0.2',
 'ObjectManagerPort': 38075,
 'ObjectStoreSocketName': '/tmp/ray/session_2024-11-04_18-21-00_595314_8/sockets/plasma_store',
 'RayletSocketName': '/tmp/ray/session_2024-11-04_18-21-00_595314_8/sockets/raylet',
 'Resources': {'memory': 4294967296.0,
               'node:10.130.0.2': 1.0,
               'node:__internal_head__': 1.0,
               'object_store_memory': 1216284672.0},
 'RuntimeEnvAgentPort': 54350,
 'alive': True}


Call a remote function on a Ray worker node that uses the GPU

In [ ]:


@ray.remote(num_returns=1, runtime_env=runtime_env, num_gpus=1, num_cpus=4)
def hello_world():
  import socket
  import torch


  return_val = f"Hello, World! I am {socket.gethostname()}"
  device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
  )
  return_val += f" Using {device} device"

  return return_val



obj_ref = hello_world.remote()
pprint(ray.get(obj_ref))

'Hello, World! I am raycluster-complete-worker-cpu-group-2vwzt Using cpu device'


In [ ]:
ray.shutdown()